================================================================================
REYNOLDS BOIDS SYSTEM - Flocking Behavior Simulation
Based on Craig Reynolds' 1986 Paper "Flocks, Herds, and Schools"
================================================================================

This notebook demonstrates emergent flocking behavior through three simple rules:
1. SEPARATION - Avoid crowding neighbors (short-range repulsion)
2. ALIGNMENT - Steer towards average heading of neighbors
3. COHESION - Steer towards average position of neighbors (long-range attraction)


WHAT YOU'LL LEARN:
1. Three simple rules that create complex flocking behavior
2. How local interactions lead to global patterns (emergence)
3. Parameter tuning and its effects on flock dynamics
4. Extensions like obstacles, predators, and goal-seeking

BOID = "Bird-oid object" - a simple agent following local rules

Historical Note: Reynolds created this for the 1987 film "Stanley and Stella
in: Breaking the Ice" and later won a Technical Academy Award for it!


In [ ]:
# ============================================================================
# CELL 1: INTRODUCTION AND IMPORTS
# ============================================================================

# Import required libraries
import numpy as np                          # For vector math
import matplotlib.pyplot as plt             # For visualization
from matplotlib.animation import FuncAnimation  # For animations
from matplotlib.patches import Circle, Wedge, FancyArrow  # For drawing
from IPython.display import HTML           # For displaying animations
import ipywidgets as widgets                # For interactive controls
from IPython.display import display        # For displaying widgets
from matplotlib.collections import LineCollection  # For efficient drawing

print("✅ Libraries imported successfully!")
print("📌 Next: Define the Boid class...")

In [ ]:
# ============================================================================
# CELL 2: THE BOID CLASS
# ============================================================================

print("\n" + "="*80)
print("PART 1: DEFINING A BOID")
print("="*80)

class Boid:
    """
    A BOID - an individual agent in the flock.

    Each boid has:
    - Position (x, y)
    - Velocity (vx, vy)
    - Perception radius (how far it can "see")

    The boid follows three simple rules based on what it perceives
    within its local neighborhood.
    """

    def __init__(self, x, y, vx, vy, perception_radius=50.0, max_speed=2.0):
        """
        Initialize a boid.

        Args:
            x, y: Initial position
            vx, vy: Initial velocity
            perception_radius: How far the boid can see neighbors
            max_speed: Maximum velocity magnitude
        """
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.array([vx, vy], dtype=float)
        self.acceleration = np.array([0.0, 0.0], dtype=float)

        self.perception_radius = perception_radius
        self.max_speed = max_speed
        self.max_force = 0.3  # Maximum steering force

    def update(self, dt=1.0):
        """
        Update position and velocity using simple physics.

        Physics:
        - velocity += acceleration * dt
        - position += velocity * dt
        - Then reset acceleration (forces are recalculated each frame)

        Args:
            dt: Time step (usually 1.0 for simplicity)
        """
        # Update velocity
        self.velocity += self.acceleration * dt

        # Limit speed to max_speed
        speed = np.linalg.norm(self.velocity)
        if speed > self.max_speed:
            self.velocity = (self.velocity / speed) * self.max_speed

        # Update position
        self.position += self.velocity * dt

        # Reset acceleration (will be recalculated next frame)
        self.acceleration = np.array([0.0, 0.0], dtype=float)

    def apply_force(self, force):
        """
        Apply a steering force to this boid.

        Multiple forces can be applied per frame - they accumulate
        in the acceleration vector.

        Args:
            force: 2D numpy array [fx, fy]
        """
        self.acceleration += force

    def get_neighbors(self, all_boids):
        """
        Find all boids within perception radius.

        This is the boid's "local neighborhood" - the only boids
        it can see and react to.

        Args:
            all_boids: List of all boids in the simulation

        Returns:
            List of Boid objects within perception_radius
        """
        neighbors = []
        for other in all_boids:
            if other is not self:  # Don't include yourself
                distance = np.linalg.norm(self.position - other.position)
                if distance < self.perception_radius:
                    neighbors.append(other)
        return neighbors

print("✅ Boid class defined!")
print("""
KEY CONCEPTS:
- Each boid is an autonomous agent
- Boids only react to nearby neighbors (local perception)
- Physics: acceleration → velocity → position
- Forces accumulate (can apply multiple steering forces)
""")


In [ ]:
# ============================================================================
# CELL 3: THE THREE RULES - SEPARATION
# ============================================================================

print("\n" + "="*80)
print("PART 2: RULE #1 - SEPARATION (Avoid Crowding)")
print("="*80)

def separation(boid, neighbors, desired_separation=25.0):
    """
    RULE 1: SEPARATION - Avoid crowding neighbors

    "Steer to avoid crowding local flockmates"

    Algorithm:
    1. For each neighbor closer than desired_separation:
       a. Calculate vector pointing away from that neighbor
       b. Weight by distance (closer = stronger repulsion)
    2. Average all avoidance vectors
    3. Convert to steering force

    This prevents boids from colliding and creates personal space.

    Args:
        boid: The boid calculating separation
        neighbors: List of nearby boids
        desired_separation: Minimum desired distance from others

    Returns:
        Steering force (2D numpy array)
    """
    steer = np.array([0.0, 0.0])
    count = 0

    # Check each neighbor
    for other in neighbors:
        distance = np.linalg.norm(boid.position - other.position)

        # If too close, calculate avoidance
        if distance < desired_separation and distance > 0:
            # Vector pointing away from neighbor
            diff = boid.position - other.position

            # Normalize and weight by distance
            # Closer neighbors have stronger influence (1/distance)
            diff = diff / distance  # Normalize
            diff = diff / distance  # Weight by inverse distance

            steer += diff
            count += 1

    # Average the avoidance vectors
    if count > 0:
        steer = steer / count

    # Convert to steering force
    if np.linalg.norm(steer) > 0:
        # Implement Reynolds' steering formula:
        # Steering = Desired - Velocity
        steer = steer / np.linalg.norm(steer) * boid.max_speed
        steer = steer - boid.velocity

        # Limit to max_force
        if np.linalg.norm(steer) > boid.max_force:
            steer = (steer / np.linalg.norm(steer)) * boid.max_force

    return steer

print("✅ Separation rule implemented!")
print("""
SEPARATION EXPLAINED:
- Creates "personal space" around each boid
- Prevents collisions
- Stronger effect when very close (1/distance weighting)
- Like pedestrians avoiding bumping into each other

Visual: Boids actively steer away from very close neighbors
Effect: Maintains spacing, prevents overlap
""")


In [ ]:
# ============================================================================
# CELL 4: THE THREE RULES - ALIGNMENT
# ============================================================================

print("\n" + "="*80)
print("PART 2: RULE #2 - ALIGNMENT (Steer Towards Average Heading)")
print("="*80)

def alignment(boid, neighbors):
    """
    RULE 2: ALIGNMENT - Steer towards average heading of neighbors

    "Steer towards the average heading of local flockmates"

    Algorithm:
    1. Calculate average velocity of all neighbors
    2. That's the desired velocity direction
    3. Steer towards it

    This makes boids move in the same direction as their neighbors,
    creating coordinated group movement.

    Args:
        boid: The boid calculating alignment
        neighbors: List of nearby boids

    Returns:
        Steering force (2D numpy array)
    """
    avg_velocity = np.array([0.0, 0.0])
    count = 0

    # Sum up all neighbor velocities
    for other in neighbors:
        avg_velocity += other.velocity
        count += 1

    if count > 0:
        # Calculate average
        avg_velocity = avg_velocity / count

        # This is our desired velocity direction
        # Scale to max_speed
        if np.linalg.norm(avg_velocity) > 0:
            avg_velocity = (avg_velocity / np.linalg.norm(avg_velocity)) * boid.max_speed

        # Steering = Desired - Current
        steer = avg_velocity - boid.velocity

        # Limit to max_force
        if np.linalg.norm(steer) > boid.max_force:
            steer = (steer / np.linalg.norm(steer)) * boid.max_force

        return steer

    return np.array([0.0, 0.0])

print("✅ Alignment rule implemented!")
print("""
ALIGNMENT EXPLAINED:
- Boids match their neighbors' velocity/heading
- Creates coordinated movement
- Like birds flying in formation
- Everyone moves in roughly the same direction

Visual: Boids turn to face the same way as neighbors
Effect: Organized, parallel movement emerges
""")

In [ ]:
# ============================================================================
# CELL 5: THE THREE RULES - COHESION
# ============================================================================

print("\n" + "="*80)
print("PART 2: RULE #3 - COHESION (Steer Towards Average Position)")
print("="*80)

def cohesion(boid, neighbors):
    """
    RULE 3: COHESION - Steer towards average position of neighbors

    "Steer to move towards the average position of local flockmates"

    Algorithm:
    1. Calculate center of mass of all neighbors
    2. Steer towards that point

    This makes boids want to stay together as a group,
    creating flocking behavior.

    Args:
        boid: The boid calculating cohesion
        neighbors: List of nearby boids

    Returns:
        Steering force (2D numpy array)
    """
    center_of_mass = np.array([0.0, 0.0])
    count = 0

    # Calculate average position of neighbors
    for other in neighbors:
        center_of_mass += other.position
        count += 1

    if count > 0:
        # Calculate average
        center_of_mass = center_of_mass / count

        # Steer towards the center
        return seek(boid, center_of_mass)

    return np.array([0.0, 0.0])

def seek(boid, target):
    """
    Helper function: Steer towards a target position.

    This is Reynolds' "seek" steering behavior.

    Args:
        boid: The boid doing the seeking
        target: Target position [x, y]

    Returns:
        Steering force
    """
    # Desired velocity: direction to target at max speed
    desired = target - boid.position

    if np.linalg.norm(desired) > 0:
        desired = (desired / np.linalg.norm(desired)) * boid.max_speed

        # Steering = Desired - Current
        steer = desired - boid.velocity

        # Limit to max_force
        if np.linalg.norm(steer) > boid.max_force:
            steer = (steer / np.linalg.norm(steer)) * boid.max_force

        return steer

    return np.array([0.0, 0.0])

print("✅ Cohesion rule implemented!")
print("""
COHESION EXPLAINED:
- Boids steer towards the center of their local group
- Keeps the flock together
- Like gravitational attraction between flock members
- Balances separation (which pushes apart)

Visual: Boids curve towards the center of nearby neighbors
Effect: Tight, cohesive groups form

THE MAGIC:
When you combine all three rules:
- SEPARATION pushes boids apart (short-range)
- COHESION pulls boids together (long-range)
- ALIGNMENT coordinates movement
→ Complex flocking behavior emerges!
""")


In [ ]:
# ============================================================================
# CELL 6: THE FLOCK CLASS
# ============================================================================

print("\n" + "="*80)
print("PART 3: THE FLOCK SIMULATION")
print("="*80)

class Flock:
    """
    FLOCK - manages a collection of boids.

    Responsibilities:
    - Store all boids
    - Apply flocking rules to each boid
    - Handle boundary conditions
    - Update all boids each time step
    """

    def __init__(self, width=800, height=600):
        """
        Initialize an empty flock.

        Args:
            width, height: Simulation boundaries
        """
        self.boids = []
        self.width = width
        self.height = height

        # Rule weights (can be adjusted for different behaviors)
        self.separation_weight = 1.5
        self.alignment_weight = 1.0
        self.cohesion_weight = 1.0

    def add_boid(self, boid):
        """Add a boid to the flock."""
        self.boids.append(boid)

    def update(self):
        """
        Update all boids for one time step.

        Algorithm:
        1. For each boid:
           a. Find neighbors
           b. Calculate separation, alignment, cohesion forces
           c. Weight and apply forces
           d. Update physics
        2. Handle boundary conditions
        """
        # Apply flocking rules to each boid
        for boid in self.boids:
            # Get local neighbors
            neighbors = boid.get_neighbors(self.boids)

            # Calculate the three forces
            sep = separation(boid, neighbors)
            ali = alignment(boid, neighbors)
            coh = cohesion(boid, neighbors)

            # Weight the forces (allows tuning behavior)
            sep *= self.separation_weight
            ali *= self.alignment_weight
            coh *= self.cohesion_weight

            # Apply forces
            boid.apply_force(sep)
            boid.apply_force(ali)
            boid.apply_force(coh)

        # Update all boid positions
        for boid in self.boids:
            boid.update()

            # Handle boundaries (wrap around)
            self.wrap_boundaries(boid)

    def wrap_boundaries(self, boid):
        """
        Wrap boids around screen edges (toroidal topology).

        When a boid goes off one edge, it appears on the opposite edge.
        This prevents boids from flying away forever.

        Args:
            boid: The boid to wrap
        """
        if boid.position[0] < 0:
            boid.position[0] = self.width
        elif boid.position[0] > self.width:
            boid.position[0] = 0

        if boid.position[1] < 0:
            boid.position[1] = self.height
        elif boid.position[1] > self.height:
            boid.position[1] = 0

print("✅ Flock class defined!")
print("""
FLOCK SYSTEM:
- Manages all boids in the simulation
- Applies the three rules each frame
- Rule weights control behavior:
  * High separation → spread out flock
  * High alignment → organized movement
  * High cohesion → tight clusters

Boundary Handling:
- Wrap-around (toroidal) - boids reappear on opposite side
- Alternative: could use repulsive forces at edges
""")


In [ ]:
# ============================================================================
# CELL 7: VISUALIZATION FUNCTION
# ============================================================================

print("\n" + "="*80)
print("PART 4: VISUALIZATION")
print("="*80)

def visualize_flock(flock, show_perception=False, show_velocity=True):
    """
    Create a single frame visualization of the flock.

    Args:
        flock: The Flock object to visualize
        show_perception: If True, draw perception circles
        show_velocity: If True, draw velocity vectors

    Returns:
        matplotlib figure
    """
    fig, ax = plt.subplots(figsize=(12, 9))

    # Set up the view
    ax.set_xlim(0, flock.width)
    ax.set_ylim(0, flock.height)
    ax.set_aspect('equal')
    ax.set_facecolor('#1a1a2e')  # Dark background
    ax.grid(False)

    # Draw each boid
    for boid in flock.boids:
        x, y = boid.position
        vx, vy = boid.velocity

        # Draw perception radius (optional)
        if show_perception:
            circle = Circle((x, y), boid.perception_radius,
                          color='cyan', alpha=0.1, fill=True)
            ax.add_patch(circle)

        # Draw boid as a triangle pointing in velocity direction
        angle = np.arctan2(vy, vx)
        size = 8

        # Triangle vertices
        triangle = np.array([
            [x + size * np.cos(angle), y + size * np.sin(angle)],
            [x + size * np.cos(angle + 2.5), y + size * np.sin(angle + 2.5)],
            [x + size * np.cos(angle - 2.5), y + size * np.sin(angle - 2.5)]
        ])

        ax.fill(triangle[:, 0], triangle[:, 1], color='yellow', alpha=0.8)

        # Draw velocity vector (optional)
        if show_velocity:
            ax.arrow(x, y, vx*3, vy*3,
                    head_width=5, head_length=5,
                    fc='red', ec='red', alpha=0.5)

    ax.set_title(f'Boids Flock Simulation - {len(flock.boids)} boids',
                 color='white', fontsize=14)
    ax.tick_params(colors='white')

    return fig

print("✅ Visualization function defined!")
print("📌 Ready to create and simulate flocks!")

In [ ]:
# ============================================================================
# CELL 8: EXPERIMENT 1 - BASIC FLOCKING
# ============================================================================

print("\n" + "="*80)
print("EXPERIMENT 1: Basic Flocking Behavior")
print("="*80)
print("""
SETUP:
- 30 boids
- Random initial positions and velocities
- Standard rule weights (1.5, 1.0, 1.0)

WATCH FOR:
- Initial chaos as boids scatter
- Gradual organization into groups
- Emergent flocking patterns
- Boids naturally avoiding each other while staying together
""")

# Create flock
flock1 = Flock(width=800, height=600)

# Add boids with random positions and velocities
np.random.seed(42)  # For reproducibility
num_boids = 30

for i in range(num_boids):
    x = np.random.uniform(100, 700)
    y = np.random.uniform(100, 500)
    vx = np.random.uniform(-2, 2)
    vy = np.random.uniform(-2, 2)

    boid = Boid(x, y, vx, vy, perception_radius=80, max_speed=2.0)
    flock1.add_boid(boid)

print(f"Created {len(flock1.boids)} boids")

# Simulate for a while to let patterns emerge
print("Simulating initial steps...")
for step in range(50):
    flock1.update()
    if step % 10 == 0:
        print(f"  Step {step}/50")
        fig1 = visualize_flock(flock1, show_perception=False, show_velocity=True)
        plt.show()

# Visualize
#print("Generating visualization...")
#fig1 = visualize_flock(flock1, show_perception=False, show_velocity=True)
#plt.show()

print("""
✅ Observe:
- Yellow triangles = boids (pointing in direction of movement)
- Red arrows = velocity vectors
- Boids form loose groups
- Movement is coordinated but not rigid
""")

In [ ]:
# ============================================================================
# CELL 9: EXPERIMENT 2 - PARAMETER VARIATIONS
# ============================================================================

print("\n" + "="*80)
print("EXPERIMENT 2: Effect of Rule Weights")
print("="*80)
print("""
We'll create three flocks with different rule weights to see
how they affect behavior:

A. High Separation → Spread out, dispersed flock
B. High Cohesion → Tight, compact groups
C. High Alignment → Organized, parallel movement
""")

# Configuration A: High Separation
print("\n--- Configuration A: High Separation ---")
flock_a = Flock(width=600, height=400)
flock_a.separation_weight = 3.0  # Very high
flock_a.alignment_weight = 1.0
flock_a.cohesion_weight = 0.5   # Low

for i in range(25):
    x = np.random.uniform(100, 500)
    y = np.random.uniform(100, 300)
    vx = np.random.uniform(-1, 1)
    vy = np.random.uniform(-1, 1)
    flock_a.add_boid(Boid(x, y, vx, vy, perception_radius=70))

for _ in range(40):
    flock_a.update()

fig_a, ax_a = plt.subplots(figsize=(10, 6))
ax_a.set_xlim(0, 600)
ax_a.set_ylim(0, 400)
ax_a.set_title('Configuration A: High Separation (3.0, 1.0, 0.5)', fontsize=12)
for boid in flock_a.boids:
    angle = np.arctan2(boid.velocity[1], boid.velocity[0])
    size = 6
    triangle = np.array([
        [boid.position[0] + size * np.cos(angle), boid.position[1] + size * np.sin(angle)],
        [boid.position[0] + size * np.cos(angle + 2.5), boid.position[1] + size * np.sin(angle + 2.5)],
        [boid.position[0] + size * np.cos(angle - 2.5), boid.position[1] + size * np.sin(angle - 2.5)]
    ])
    ax_a.fill(triangle[:, 0], triangle[:, 1], color='red', alpha=0.7)
plt.show()

print("Result: Boids spread out, maintain distance")

# Configuration B: High Cohesion
print("\n--- Configuration B: High Cohesion ---")
flock_b = Flock(width=600, height=400)
flock_b.separation_weight = 1.0
flock_b.alignment_weight = 1.0
flock_b.cohesion_weight = 3.0  # Very high

for i in range(25):
    x = np.random.uniform(100, 500)
    y = np.random.uniform(100, 300)
    vx = np.random.uniform(-1, 1)
    vy = np.random.uniform(-1, 1)
    flock_b.add_boid(Boid(x, y, vx, vy, perception_radius=70))

for _ in range(40):
    flock_b.update()

fig_b, ax_b = plt.subplots(figsize=(10, 6))
ax_b.set_xlim(0, 600)
ax_b.set_ylim(0, 400)
ax_b.set_title('Configuration B: High Cohesion (1.0, 1.0, 3.0)', fontsize=12)
for boid in flock_b.boids:
    angle = np.arctan2(boid.velocity[1], boid.velocity[0])
    size = 6
    triangle = np.array([
        [boid.position[0] + size * np.cos(angle), boid.position[1] + size * np.sin(angle)],
        [boid.position[0] + size * np.cos(angle + 2.5), boid.position[1] + size * np.sin(angle + 2.5)],
        [boid.position[0] + size * np.cos(angle - 2.5), boid.position[1] + size * np.sin(angle - 2.5)]
    ])
    ax_b.fill(triangle[:, 0], triangle[:, 1], color='green', alpha=0.7)
plt.show()

print("Result: Boids cluster tightly together")

# Configuration C: High Alignment
print("\n--- Configuration C: High Alignment ---")
flock_c = Flock(width=600, height=400)
flock_c.separation_weight = 1.0
flock_c.alignment_weight = 3.0  # Very high
flock_c.cohesion_weight = 1.0

for i in range(25):
    x = np.random.uniform(100, 500)
    y = np.random.uniform(100, 300)
    vx = np.random.uniform(-1, 1)
    vy = np.random.uniform(-1, 1)
    flock_c.add_boid(Boid(x, y, vx, vy, perception_radius=70))

for _ in range(40):
    flock_c.update()

fig_c, ax_c = plt.subplots(figsize=(10, 6))
ax_c.set_xlim(0, 600)
ax_c.set_ylim(0, 400)
ax_c.set_title('Configuration C: High Alignment (1.0, 3.0, 1.0)', fontsize=12)
for boid in flock_c.boids:
    angle = np.arctan2(boid.velocity[1], boid.velocity[0])
    size = 6
    triangle = np.array([
        [boid.position[0] + size * np.cos(angle), boid.position[1] + size * np.sin(angle)],
        [boid.position[0] + size * np.cos(angle + 2.5), boid.position[1] + size * np.sin(angle + 2.5)],
        [boid.position[0] + size * np.cos(angle - 2.5), boid.position[1] + size * np.sin(angle - 2.5)]
    ])
    ax_c.fill(triangle[:, 0], triangle[:, 1], color='blue', alpha=0.7)
plt.show()

print("Result: Boids move in parallel, organized formations")

print("""
✅ INSIGHTS:
- Rule weights dramatically affect flock behavior
- Real flocks (birds, fish) likely balance all three
- Different species may have different "natural" weights
- Tuning creates different emergent patterns
""")


In [ ]:
# ============================================================================
# CELL 10: EXPERIMENT 3 - OBSTACLE AVOIDANCE
# ============================================================================

print("\n" + "="*80)
print("EXPERIMENT 3: Adding Obstacles")
print("="*80)
print("""
EXTENSION: Boids avoid obstacles in the environment

New behavior: Avoid obstacles (circular regions)
- Similar to separation, but from static objects
- Boids steer away when approaching obstacles
""")

class Obstacle:
    """A circular obstacle that boids must avoid."""
    def __init__(self, x, y, radius):
        self.position = np.array([x, y], dtype=float)
        self.radius = radius

def avoid_obstacles(boid, obstacles, avoidance_distance=80):
    """
    Steer away from obstacles.

    Similar to separation, but for static obstacles.

    Args:
        boid: The boid avoiding obstacles
        obstacles: List of Obstacle objects
        avoidance_distance: How far to detect obstacles

    Returns:
        Steering force
    """
    steer = np.array([0.0, 0.0])

    for obstacle in obstacles:
        distance = np.linalg.norm(boid.position - obstacle.position)
        distance = distance - obstacle.radius  # Distance to surface

        if distance < avoidance_distance and distance > 0:
            # Vector away from obstacle
            diff = boid.position - obstacle.position
            diff = diff / np.linalg.norm(diff)  # Normalize
            diff = diff / distance  # Weight by inverse distance

            steer += diff

    # Convert to steering force
    if np.linalg.norm(steer) > 0:
        steer = (steer / np.linalg.norm(steer)) * boid.max_speed
        steer = steer - boid.velocity

        if np.linalg.norm(steer) > boid.max_force:
            steer = (steer / np.linalg.norm(steer)) * boid.max_force

    return steer

# Create flock with obstacles
flock_obs = Flock(width=800, height=600)

# Add obstacles
obstacles = [
    Obstacle(300, 300, 60),
    Obstacle(600, 200, 50),
    Obstacle(400, 450, 70)
]

# Add boids
for i in range(40):
    x = np.random.uniform(50, 750)
    y = np.random.uniform(50, 550)
    vx = np.random.uniform(-2, 2)
    vy = np.random.uniform(-2, 2)
    flock_obs.add_boid(Boid(x, y, vx, vy, perception_radius=80))

# Simulate with obstacle avoidance
print("Simulating with obstacles...")
for step in range(60):
    for boid in flock_obs.boids:
        neighbors = boid.get_neighbors(flock_obs.boids)

        # Standard flocking rules
        sep = separation(boid, neighbors) * flock_obs.separation_weight
        ali = alignment(boid, neighbors) * flock_obs.alignment_weight
        coh = cohesion(boid, neighbors) * flock_obs.cohesion_weight

        # Add obstacle avoidance
        obs_avoid = avoid_obstacles(boid, obstacles) * 2.0  # Strong avoidance

        boid.apply_force(sep)
        boid.apply_force(ali)
        boid.apply_force(coh)
        boid.apply_force(obs_avoid)

    for boid in flock_obs.boids:
        boid.update()
        flock_obs.wrap_boundaries(boid)

    # Visualize with obstacles
    fig_obs, ax_obs = plt.subplots(figsize=(12, 9))
    ax_obs.set_xlim(0, 800)
    ax_obs.set_ylim(0, 600)
    ax_obs.set_aspect('equal')
    ax_obs.set_facecolor('#1a1a2e')

    # Draw obstacles
    for obs in obstacles:
        circle = Circle(obs.position, obs.radius, color='red', alpha=0.6)
        ax_obs.add_patch(circle)

    # Draw boids
    for boid in flock_obs.boids:
        angle = np.arctan2(boid.velocity[1], boid.velocity[0])
        size = 8
        triangle = np.array([
            [boid.position[0] + size * np.cos(angle), boid.position[1] + size * np.sin(angle)],
            [boid.position[0] + size * np.cos(angle + 2.5), boid.position[1] + size * np.sin(angle + 2.5)],
            [boid.position[0] + size * np.cos(angle - 2.5), boid.position[1] + size * np.sin(angle - 2.5)]
        ])
        ax_obs.fill(triangle[:, 0], triangle[:, 1], color='yellow', alpha=0.8)

    ax_obs.set_title('Boids with Obstacle Avoidance', color='white', fontsize=14)
    plt.show()

print("""
✅ Result: Boids navigate around obstacles while maintaining flock
- Red circles = obstacles
- Boids naturally flow around them
- Flocking behavior preserved while avoiding collisions
""")


In [ ]:
# ============================================================================
# CELL 11: EXPERIMENT 4 - PREDATOR AND PREY
# ============================================================================

print("\n" + "="*80)
print("EXPERIMENT 4: Predator-Prey Dynamics")
print("="*80)
print("""
EXTENSION: Add a predator that chases boids

Predator behavior:
- Seeks the nearest boid
- Boids flee from predator when nearby

This creates dramatic emergent behavior!
""")

class Predator(Boid):
    """A predator that hunts boids."""
    def __init__(self, x, y):
        super().__init__(x, y, 0, 0, perception_radius=150, max_speed=2.5)
        self.max_force = 0.5

    def hunt(self, boids):
        """
        Chase the nearest boid.

        Args:
            boids: List of prey boids
        """
        if not boids:
            return

        # Find nearest boid
        nearest = None
        min_dist = float('inf')

        for boid in boids:
            dist = np.linalg.norm(self.position - boid.position)
            if dist < min_dist:
                min_dist = dist
                nearest = boid

        # Chase it
        if nearest:
            seek_force = seek(self, nearest.position)
            self.apply_force(seek_force)

def flee(boid, predators, flee_distance=100):
    """
    Flee from nearby predators.

    Args:
        boid: The prey boid
        predators: List of Predator objects
        flee_distance: How far to detect predators

    Returns:
        Steering force away from predators
    """
    steer = np.array([0.0, 0.0])

    for predator in predators:
        distance = np.linalg.norm(boid.position - predator.position)

        if distance < flee_distance:
            # Vector away from predator
            diff = boid.position - predator.position

            if np.linalg.norm(diff) > 0:
                diff = diff / np.linalg.norm(diff)
                # Stronger when closer
                diff = diff / (distance / flee_distance)
                steer += diff

    # Convert to steering force
    if np.linalg.norm(steer) > 0:
        steer = (steer / np.linalg.norm(steer)) * boid.max_speed
        steer = steer - boid.velocity

        if np.linalg.norm(steer) > boid.max_force:
            steer = (steer / np.linalg.norm(steer)) * boid.max_force

    return steer

# Create flock with predator
flock_pred = Flock(width=800, height=600)

# Add prey boids
for i in range(50):
    x = np.random.uniform(100, 700)
    y = np.random.uniform(100, 500)
    vx = np.random.uniform(-1, 1)
    vy = np.random.uniform(-1, 1)
    flock_pred.add_boid(Boid(x, y, vx, vy, perception_radius=60))

# Add predator
predator = Predator(400, 300)

print("Simulating predator-prey interaction...")
for step in range(80):
    # Update predator
    predator.hunt(flock_pred.boids)
    predator.update()
    flock_pred.wrap_boundaries(predator)

    # Update prey
    for boid in flock_pred.boids:
        neighbors = boid.get_neighbors(flock_pred.boids)

        # Standard flocking
        sep = separation(boid, neighbors) * flock_pred.separation_weight
        ali = alignment(boid, neighbors) * flock_pred.alignment_weight
        coh = cohesion(boid, neighbors) * flock_pred.cohesion_weight

        # FLEE from predator (highest priority!)
        flee_force = flee(boid, [predator]) * 3.0

        boid.apply_force(sep)
        boid.apply_force(ali)
        boid.apply_force(coh)
        boid.apply_force(flee_force)

        boid.update()
        flock_pred.wrap_boundaries(boid)

    # Visualize
    fig_pred, ax_pred = plt.subplots(figsize=(12, 9))
    ax_pred.set_xlim(0, 800)
    ax_pred.set_ylim(0, 600)
    ax_pred.set_aspect('equal')
    ax_pred.set_facecolor('#1a1a2e')

    # Draw prey boids
    for boid in flock_pred.boids:
        angle = np.arctan2(boid.velocity[1], boid.velocity[0])
        size = 6
        triangle = np.array([
            [boid.position[0] + size * np.cos(angle), boid.position[1] + size * np.sin(angle)],
            [boid.position[0] + size * np.cos(angle + 2.5), boid.position[1] + size * np.sin(angle + 2.5)],
            [boid.position[0] + size * np.cos(angle - 2.5), boid.position[1] + size * np.sin(angle - 2.5)]
        ])
        ax_pred.fill(triangle[:, 0], triangle[:, 1], color='cyan', alpha=0.7)

    # Draw predator (larger, different color)
    pred_angle = np.arctan2(predator.velocity[1], predator.velocity[0])
    pred_size = 12
    pred_triangle = np.array([
        [predator.position[0] + pred_size * np.cos(pred_angle), predator.position[1] + pred_size * np.sin(pred_angle)],
        [predator.position[0] + pred_size * np.cos(pred_angle + 2.5), predator.position[1] + pred_size * np.sin(pred_angle + 2.5)],
        [predator.position[0] + pred_size * np.cos(pred_angle - 2.5), predator.position[1] + pred_size * np.sin(pred_angle - 2.5)]
    ])
    ax_pred.fill(pred_triangle[:, 0], pred_triangle[:, 1], color='red', alpha=0.9)

    # Draw predator's perception radius
    pred_circle = Circle(predator.position, predator.perception_radius,
                        color='red', alpha=0.1, fill=True)
    ax_pred.add_patch(pred_circle)

    ax_pred.set_title('Predator-Prey Dynamics', color='white', fontsize=14)
    plt.show()

print("""
✅ Dramatic emergent behavior!
- Cyan = prey boids
- Red = predator
- Boids scatter when predator approaches
- Flock reforms when predator passes
- Creates natural-looking escape behavior
""")


In [ ]:
# ============================================================================
# CELL 12: INTERACTIVE PARAMETER TUNING
# ============================================================================

print("\n" + "="*80)
print("PART 5: INTERACTIVE EXPERIMENTATION")
print("="*80)
print("""
Use the sliders below to experiment with rule weights in real-time!

Adjust the three weights and see how the flock behavior changes.
""")

def create_interactive_boids():
    """Create interactive widget for experimenting with boid parameters."""

    # Sliders for rule weights
    sep_slider = widgets.FloatSlider(
        value=1.5, min=0, max=5, step=0.1,
        description='Separation:', style={'description_width': '120px'}
    )
    ali_slider = widgets.FloatSlider(
        value=1.0, min=0, max=5, step=0.1,
        description='Alignment:', style={'description_width': '120px'}
    )
    coh_slider = widgets.FloatSlider(
        value=1.0, min=0, max=5, step=0.1,
        description='Cohesion:', style={'description_width': '120px'}
    )

    num_boids_slider = widgets.IntSlider(
        value=30, min=10, max=100, step=5,
        description='Num Boids:', style={'description_width': '120px'}
    )

    perception_slider = widgets.FloatSlider(
        value=70, min=30, max=150, step=10,
        description='Perception:', style={'description_width': '120px'}
    )

    run_button = widgets.Button(
        description='Run Simulation',
        button_style='success'
    )

    output = widgets.Output()

    def on_run(b):
        with output:
            output.clear_output(wait=True)

            print(f"Simulating with weights: Sep={sep_slider.value}, "
                  f"Ali={ali_slider.value}, Coh={coh_slider.value}")

            # Create new flock
            flock = Flock(width=700, height=500)
            flock.separation_weight = sep_slider.value
            flock.alignment_weight = ali_slider.value
            flock.cohesion_weight = coh_slider.value

            # Add boids
            for i in range(num_boids_slider.value):
                x = np.random.uniform(50, 650)
                y = np.random.uniform(50, 450)
                vx = np.random.uniform(-2, 2)
                vy = np.random.uniform(-2, 2)
                boid = Boid(x, y, vx, vy,
                           perception_radius=perception_slider.value,
                           max_speed=2.0)
                flock.add_boid(boid)

            # Simulate
            for step in range(50):
                flock.update()

            # Visualize
            fig, ax = plt.subplots(figsize=(11, 8))
            ax.set_xlim(0, 700)
            ax.set_ylim(0, 500)
            ax.set_aspect('equal')
            ax.set_facecolor('#1a1a2e')
            ax.grid(False)

            for boid in flock.boids:
                angle = np.arctan2(boid.velocity[1], boid.velocity[0])
                size = 7
                triangle = np.array([
                    [boid.position[0] + size * np.cos(angle),
                     boid.position[1] + size * np.sin(angle)],
                    [boid.position[0] + size * np.cos(angle + 2.5),
                     boid.position[1] + size * np.sin(angle + 2.5)],
                    [boid.position[0] + size * np.cos(angle - 2.5),
                     boid.position[1] + size * np.sin(angle - 2.5)]
                ])
                ax.fill(triangle[:, 0], triangle[:, 1], color='yellow', alpha=0.8)

            ax.set_title(f'Interactive Boids - Weights: ({sep_slider.value}, '
                        f'{ali_slider.value}, {coh_slider.value})',
                        color='white', fontsize=12)
            plt.show()

            print("✅ Simulation complete! Try different parameters.")

    run_button.on_click(on_run)

    # Layout
    controls = widgets.VBox([
        widgets.HTML("<h3>🎮 Boids Parameter Explorer</h3>"),
        sep_slider,
        ali_slider,
        coh_slider,
        num_boids_slider,
        perception_slider,
        run_button,
        output
    ])

    display(controls)

create_interactive_boids()


In [ ]:
# ============================================================================
# CELL 15: BONUS - CREATE AN ANIMATION
# ============================================================================

print("\n" + "="*80)
print("BONUS: Creating an Animation")
print("="*80)
print("""
Run this cell to create an animated visualization of the flock over time.
This may take a minute to generate...
""")

def create_animation(num_frames=100, num_boids=40):
    """
    Create an animated visualization of boid flocking.

    Args:
        num_frames: Number of animation frames
        num_boids: Number of boids in the flock

    Returns:
        HTML animation
    """
    # Create flock
    flock = Flock(width=600, height=400)

    for i in range(num_boids):
        x = np.random.uniform(50, 550)
        y = np.random.uniform(50, 350)
        vx = np.random.uniform(-2, 2)
        vy = np.random.uniform(-2, 2)
        flock.add_boid(Boid(x, y, vx, vy, perception_radius=70))

    # Set up the figure
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.set_xlim(0, 600)
    ax.set_ylim(0, 400)
    ax.set_aspect('equal')
    ax.set_facecolor('#1a1a2e')
    ax.grid(False)

    # Initialize plot elements
    triangles = []
    for _ in flock.boids:
        triangle = ax.fill([], [], color='yellow', alpha=0.8)[0]
        triangles.append(triangle)

    title = ax.text(300, 380, '', ha='center', color='white', fontsize=12)

    def init():
        for triangle in triangles:
            triangle.set_xy(np.array([[0, 0], [0, 0], [0, 0]]))
        title.set_text('')
        return triangles + [title]

    def update(frame):
        # Update flock
        flock.update()

        # Update visualization
        for i, boid in enumerate(flock.boids):
            angle = np.arctan2(boid.velocity[1], boid.velocity[0])
            size = 7
            triangle_coords = np.array([
                [boid.position[0] + size * np.cos(angle),
                 boid.position[1] + size * np.sin(angle)],
                [boid.position[0] + size * np.cos(angle + 2.5),
                 boid.position[1] + size * np.sin(angle + 2.5)],
                [boid.position[0] + size * np.cos(angle - 2.5),
                 boid.position[1] + size * np.sin(angle - 2.5)]
            ])
            triangles[i].set_xy(triangle_coords)

        title.set_text(f'Boids Flocking Animation - Frame {frame}/{num_frames}')

        return triangles + [title]

    anim = FuncAnimation(fig, update, frames=num_frames, init_func=init,
                        blit=True, interval=50, repeat=True)

    plt.close()
    return HTML(anim.to_jshtml())

print("Generating animation...")
animation = create_animation(num_frames=100, num_boids=35)
display(animation)

print("\n✅ Animation complete!")
print("Watch how the flock self-organizes over time.")
print("Notice the emergent patterns - no boid is in charge!")

In [ ]:
# ============================================================================
# CELL 16: CROWD SIMULATION - PEDESTRIAN DYNAMICS
# ============================================================================

print("\n" + "="*80)
print("ADVANCED APPLICATION: Crowd Animation")
print("="*80)
print("""
REAL-WORLD APPLICATION: Simulating pedestrian crowd behavior

This extends Boids to simulate humans in crowds:
- Multiple goals (exits, destinations)
- Social forces (personal space)
- Group behavior (families stay together)
- Obstacle navigation (buildings, walls)

Used in: Architecture, urban planning, evacuation simulation, game AI
""")

class Pedestrian(Boid):
    """
    A pedestrian agent with human-like behavior.

    Extensions from basic Boid:
    - Has a specific goal/destination
    - Larger personal space (social distancing)
    - Slower speeds
    - Can be part of a group
    """
    def __init__(self, x, y, goal, group_id=None):
        """
        Initialize a pedestrian.

        Args:
            x, y: Starting position
            goal: Target destination (x, y)
            group_id: If not None, this pedestrian belongs to a group
        """
        # Random initial velocity
        vx = np.random.uniform(-0.5, 0.5)
        vy = np.random.uniform(-0.5, 0.5)

        super().__init__(x, y, vx, vy,
                        perception_radius=40.0,  # How far they see others
                        max_speed=1.5)           # Walking speed

        self.goal = np.array(goal, dtype=float)
        self.group_id = group_id
        self.personal_space = 15.0  # Preferred distance from others
        self.max_force = 0.2  # Humans turn gradually
        self.goal_weight = 2.0  # Strong motivation to reach goal

    def is_at_goal(self, threshold=10.0):
        """Check if pedestrian has reached their goal."""
        return np.linalg.norm(self.position - self.goal) < threshold

def pedestrian_separation(ped, neighbors):
    """
    Social force model - humans maintain personal space.

    Similar to boid separation, but:
    - Uses personal_space as desired distance
    - Stronger repulsion (humans really don't like being too close)

    Args:
        ped: The pedestrian
        neighbors: List of nearby pedestrians

    Returns:
        Steering force
    """
    steer = np.array([0.0, 0.0])
    count = 0

    for other in neighbors:
        distance = np.linalg.norm(ped.position - other.position)

        # Only avoid if violating personal space
        if distance < ped.personal_space and distance > 0:
            # Vector away from other person
            diff = ped.position - other.position
            diff = diff / distance  # Normalize

            # Weight by how much personal space is violated
            violation = (ped.personal_space - distance) / ped.personal_space
            diff = diff * violation * 2.0  # Strong repulsion

            steer += diff
            count += 1

    if count > 0:
        steer = steer / count

    # Convert to steering force
    if np.linalg.norm(steer) > 0:
        steer = (steer / np.linalg.norm(steer)) * ped.max_speed
        steer = steer - ped.velocity

        if np.linalg.norm(steer) > ped.max_force:
            steer = (steer / np.linalg.norm(steer)) * ped.max_force

    return steer

def group_cohesion(ped, neighbors):
    """
    Keep groups together (families, friends).

    Pedestrians in the same group want to stay near each other.

    Args:
        ped: The pedestrian
        neighbors: List of nearby pedestrians

    Returns:
        Steering force toward group members
    """
    if ped.group_id is None:
        return np.array([0.0, 0.0])

    group_center = np.array([0.0, 0.0])
    count = 0

    # Find group members
    for other in neighbors:
        if other.group_id == ped.group_id:
            group_center += other.position
            count += 1

    if count > 0:
        group_center = group_center / count
        # Steer toward group center (but not too strongly)
        return seek(ped, group_center) * 0.5

    return np.array([0.0, 0.0])

def goal_seeking(ped):
    """
    Navigate toward goal destination.

    This is the primary motivation for pedestrians.

    Args:
        ped: The pedestrian

    Returns:
        Steering force toward goal
    """
    if ped.is_at_goal():
        # Slow down when at goal
        return -ped.velocity * 0.5

    return seek(ped, ped.goal)

class CrowdSimulation:
    """
    Manages a crowd of pedestrians navigating an environment.
    """
    def __init__(self, width=1000, height=800):
        self.pedestrians = []
        self.obstacles = []  # Buildings, walls, etc.
        self.width = width
        self.height = height

    def add_pedestrian(self, ped):
        self.pedestrians.append(ped)

    def add_obstacle(self, obstacle):
        self.obstacles.append(obstacle)

    def update(self):
        """Update all pedestrians for one time step."""
        for ped in self.pedestrians:
            # Skip if already at goal
            if ped.is_at_goal():
                ped.velocity *= 0.9  # Gradually stop
                continue

            # Get nearby pedestrians
            neighbors = []
            for other in self.pedestrians:
                if other is not ped:
                    dist = np.linalg.norm(ped.position - other.position)
                    if dist < ped.perception_radius:
                        neighbors.append(other)

            # Calculate forces
            goal_force = goal_seeking(ped) * ped.goal_weight
            separation_force = pedestrian_separation(ped, neighbors) * 2.5
            group_force = group_cohesion(ped, neighbors) * 1.0
            obstacle_force = avoid_obstacles(ped, self.obstacles,
                                            avoidance_distance=60) * 3.0

            # Apply forces
            ped.apply_force(goal_force)
            ped.apply_force(separation_force)
            ped.apply_force(group_force)
            ped.apply_force(obstacle_force)

            # Update physics
            ped.update()

            # Keep in bounds
            ped.position[0] = np.clip(ped.position[0], 0, self.width)
            ped.position[1] = np.clip(ped.position[1], 0, self.height)






In [ ]:
# ============================================================================
# SCENARIO 1: EVACUATION SIMULATION
# ============================================================================

print("\n--- Scenario 1: Building Evacuation ---")
print("""
Simulating people evacuating a building through two exits.

Setup:
- 80 pedestrians starting in a room
- Two exit doors
- Central obstacle (pillar)
- People form lanes naturally
- Groups try to stay together
""")

# Create simulation
evac_sim = CrowdSimulation(width=800, height=600)

# Add central pillar obstacle
evac_sim.add_obstacle(Obstacle(400, 300, 60))

# Add walls as obstacles (approximate with circles)
for x in range(50, 750, 80):
    evac_sim.add_obstacle(Obstacle(x, 50, 30))  # Top wall
    evac_sim.add_obstacle(Obstacle(x, 550, 30))  # Bottom wall

# Define exits
exit_left = (100, 300)
exit_right = (700, 300)

# Add pedestrians
np.random.seed(123)
num_groups = 8
group_size = 5
current_group = 0

for i in range(num_groups * group_size):
    # Start in center area
    x = np.random.uniform(250, 550)
    y = np.random.uniform(200, 400)

    # Assign to group
    group_id = current_group
    if (i + 1) % group_size == 0:
        current_group += 1

    # Alternate between exits
    goal = exit_left if i % 2 == 0 else exit_right

    ped = Pedestrian(x, y, goal, group_id=group_id)
    evac_sim.add_pedestrian(ped)

# Add some solo pedestrians
for i in range(40):
    x = np.random.uniform(250, 550)
    y = np.random.uniform(200, 400)
    goal = exit_left if i % 2 == 0 else exit_right
    ped = Pedestrian(x, y, goal, group_id=None)
    evac_sim.add_pedestrian(ped)

print(f"Created {len(evac_sim.pedestrians)} pedestrians")
print("Simulating evacuation...")

# Simulate
for step in range(220):
    evac_sim.update()
    if step % 30 == 0:
        at_goal = sum(1 for p in evac_sim.pedestrians if p.is_at_goal())
        print(f"  Step {step}/120: {at_goal} people evacuated")

        # Visualize
        fig_evac, ax_evac = plt.subplots(figsize=(14, 10))
        ax_evac.set_xlim(0, 800)
        ax_evac.set_ylim(0, 600)
        ax_evac.set_aspect('equal')
        ax_evac.set_facecolor('#2c3e50')

        # Draw obstacles (building structure)
        for obs in evac_sim.obstacles:
            circle = Circle(obs.position, obs.radius, color='#34495e', alpha=0.8)
            ax_evac.add_patch(circle)

        # Draw exits (green)
        exit_size = 40
        ax_evac.add_patch(Circle(exit_left, exit_size, color='green', alpha=0.6))
        ax_evac.add_patch(Circle(exit_right, exit_size, color='green', alpha=0.6))
        ax_evac.text(exit_left[0], exit_left[1] - 60, 'EXIT',
                    ha='center', color='white', fontsize=12, weight='bold')
        ax_evac.text(exit_right[0], exit_right[1] - 60, 'EXIT',
                    ha='center', color='white', fontsize=12, weight='bold')

        # Draw pedestrians
        group_colors = plt.cm.Set3(np.linspace(0, 1, num_groups))

        for ped in evac_sim.pedestrians:
            x, y = ped.position

            # Color by group
            if ped.group_id is not None and ped.group_id < num_groups:
                color = group_colors[ped.group_id]
            else:
                color = 'white'

            # Draw as circle (top-down view)
            if ped.is_at_goal():
                # Faded if at goal
                circle = Circle((x, y), 6, color=color, alpha=0.3)
            else:
                circle = Circle((x, y), 6, color=color, alpha=0.8)

            ax_evac.add_patch(circle)

            # Draw direction indicator (small line)
            if not ped.is_at_goal() and np.linalg.norm(ped.velocity) > 0.1:
                angle = np.arctan2(ped.velocity[1], ped.velocity[0])
                dx = 10 * np.cos(angle)
                dy = 10 * np.sin(angle)
                ax_evac.arrow(x, y, dx, dy, head_width=4, head_length=4,
                            fc=color, ec=color, alpha=0.6, linewidth=2)

        ax_evac.set_title('Evacuation Simulation - Crowd Dynamics',
                        color='white', fontsize=16, weight='bold')
        ax_evac.text(400, 580, 'Groups shown in same colors - notice how they stay together',
                    ha='center', color='white', fontsize=10)
        ax_evac.axis('off')

        plt.tight_layout()
        plt.show()

at_goal_final = sum(1 for p in evac_sim.pedestrians if p.is_at_goal())
print(f"\n✅ Evacuation result: {at_goal_final}/{len(evac_sim.pedestrians)} evacuated")
print("""
OBSERVATIONS:
- People naturally form lanes toward exits
- Groups (same color) stay together
- Personal space is maintained (no overlap)
- Bottlenecks form at exits (realistic!)
- Flow around obstacles emerges naturally
""")


In [ ]:
# ============================================================================
# SCENARIO 2: STREET CROSSING - BIDIRECTIONAL FLOW
# ============================================================================

print("\n--- Scenario 2: Street Crossing ---")
print("""
Simulating pedestrians crossing paths in opposite directions.

Setup:
- Two groups moving in opposite directions
- Must navigate through each other
- Lane formation emerges naturally
""")

# Create simulation
street_sim = CrowdSimulation(width=800, height=400)

# Group 1: Moving left to right (bottom half)
for i in range(30):
    x = np.random.uniform(50, 150)
    y = np.random.uniform(180, 220)
    goal = (750, 200)
    ped = Pedestrian(x, y, goal, group_id=0)
    street_sim.add_pedestrian(ped)

# Group 2: Moving right to left (top half)
for i in range(30):
    x = np.random.uniform(650, 750)
    y = np.random.uniform(180, 220)
    goal = (50, 200)
    ped = Pedestrian(x, y, goal, group_id=1)
    street_sim.add_pedestrian(ped)

print("Simulating bidirectional flow...")
for step in range(450):
    street_sim.update()
    if step % 20 == 0:
      # Visualize
      fig_street, ax_street = plt.subplots(figsize=(14, 7))
      ax_street.set_xlim(0, 800)
      ax_street.set_ylim(0, 400)
      ax_street.set_aspect('equal')
      ax_street.set_facecolor('#34495e')

      # Draw pedestrians
      for ped in street_sim.pedestrians:
          x, y = ped.position

          # Color by direction
          if ped.group_id == 0:
              color = '#3498db'  # Blue - moving right
          else:
              color = '#e74c3c'  # Red - moving left

          circle = Circle((x, y), 6, color=color, alpha=0.7)
          ax_street.add_patch(circle)

          # Draw velocity
          if np.linalg.norm(ped.velocity) > 0.1:
              angle = np.arctan2(ped.velocity[1], ped.velocity[0])
              dx = 12 * np.cos(angle)
              dy = 12 * np.sin(angle)
              ax_street.arrow(x, y, dx, dy, head_width=5, head_length=5,
                            fc=color, ec=color, alpha=0.5, linewidth=2)

      # Add legend
      ax_street.text(100, 350, '→ Moving Right', color='#3498db',
                    fontsize=12, weight='bold')
      ax_street.text(700, 350, '← Moving Left', color='#e74c3c',
                    fontsize=12, weight='bold')

      ax_street.set_title('Bidirectional Pedestrian Flow - Lane Formation',
                        color='white', fontsize=16, weight='bold')
      ax_street.axis('off')

      plt.tight_layout()
      plt.show()

print("""
✅ EMERGENT BEHAVIOR:
- Lanes form naturally (no explicit programming!)
- Groups separate to minimize conflicts
- Similar to real pedestrian behavior on sidewalks
- This is called "self-organizing lane formation"
- Observed in real crowds worldwide
""")


In [ ]:
# ============================================================================
# SCENARIO 3: CONCERT/EVENT CROWD
# ============================================================================

print("\n--- Scenario 3: Concert Venue ---")
print("""
Simulating crowd gathering at a concert venue.

Setup:
- People arrive from multiple entrances
- Converge toward stage area
- Dense crowd near stage
- More spacing near entrances
""")

# Create simulation
concert_sim = CrowdSimulation(width=1000, height=700)

# Define stage area (top center)
stage_center = (500, 100)
stage_width = 200

# Add stage as obstacles (approximate with circles)
for x in range(350, 650, 50):
    concert_sim.add_obstacle(Obstacle(x, 80, 25))

# Define entrance areas
entrance_left = (150, 650)
entrance_center = (500, 650)
entrance_right = (850, 650)

# Add pedestrians entering from different points
np.random.seed(456)

for i in range(120):
    # Random entrance
    entrance = [entrance_left, entrance_center, entrance_right][i % 3]

    # Start near entrance with some spread
    x = entrance[0] + np.random.uniform(-50, 50)
    y = entrance[1] + np.random.uniform(-30, 0)

    # Goal: somewhere near the stage
    goal_x = stage_center[0] + np.random.uniform(-stage_width, stage_width)
    goal_y = np.random.uniform(150, 350)  # Various distances from stage

    # Some people in groups
    group_id = i // 4 if i < 80 else None

    ped = Pedestrian(x, y, (goal_x, goal_y), group_id=group_id)
    concert_sim.add_pedestrian(ped)

print("Simulating crowd gathering...")
for step in range(1000):
    concert_sim.update()
    if step % 100 == 0:
        arrived = sum(1 for p in concert_sim.pedestrians if p.is_at_goal())
        print(f"  Step {step}/100: {arrived} people at their spots")

        # Visualize with density heatmap effect
        fig_concert, ax_concert = plt.subplots(figsize=(15, 11))
        ax_concert.set_xlim(0, 1000)
        ax_concert.set_ylim(0, 700)
        ax_concert.set_aspect('equal')
        ax_concert.set_facecolor('#1a1a1a')

        # Draw stage
        stage_rect = plt.Rectangle((350, 50), 300, 60,
                                  color='#e74c3c', alpha=0.7)
        ax_concert.add_patch(stage_rect)
        ax_concert.text(500, 80, 'STAGE', ha='center', va='center',
                      color='white', fontsize=16, weight='bold')

        # Draw obstacles
        for obs in concert_sim.obstacles:
            circle = Circle(obs.position, obs.radius, color='#555', alpha=0.6)
            ax_concert.add_patch(circle)

        # Draw entrances
        for entrance, label in [(entrance_left, 'Entrance 1'),
                                (entrance_center, 'Entrance 2'),
                                (entrance_right, 'Entrance 3')]:
            ax_concert.plot(entrance[0], entrance[1], 'g^', markersize=20, alpha=0.7)
            ax_concert.text(entrance[0], entrance[1] + 25, label,
                          ha='center', color='lightgreen', fontsize=10)

        # Draw pedestrians with size based on proximity to stage (depth)
        for ped in concert_sim.pedestrians:
            x, y = ped.position

            # Size based on y-position (perspective effect)
            size = 3 + (700 - y) / 100

            # Color based on group
            if ped.group_id is not None and ped.group_id < 20:
                color = plt.cm.hsv(ped.group_id / 20)
            else:
                color = 'yellow'

            # Brighter if at goal
            alpha = 0.9 if ped.is_at_goal() else 0.6

            circle = Circle((x, y), size, color=color, alpha=alpha)
            ax_concert.add_patch(circle)

        ax_concert.set_title('Concert Venue - Crowd Gathering Simulation',
                            color='white', fontsize=18, weight='bold')
        ax_concert.text(500, 680,
                      'Notice: Dense crowd near stage, natural spacing emerges',
                      ha='center', color='white', fontsize=11)
        ax_concert.axis('off')

        plt.tight_layout()
        plt.show()

arrived = sum(1 for p in concert_sim.pedestrians if p.is_at_goal())
print(f"\n✅ Crowd formation: {arrived}/{len(concert_sim.pedestrians)} at destination")
print("""
REALISTIC FEATURES:
- Density gradient (dense near stage, sparse at back)
- Groups stay together while moving
- Natural flow patterns emerge
- No collisions despite high density
- Self-organized spatial distribution

APPLICATIONS:
🏟️ Venue design and capacity planning
🚨 Emergency evacuation planning
🎮 Game crowd AI (sports games, open worlds)
🎬 Film/animation (realistic background crowds)
🏙️ Urban planning and architecture
""")